In [ ]:
pip install dash pandas

In [267]:
pip install dash


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ipywidgets


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import dash
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.express as px
import pandas as pd
from threading import Timer  # end server

# Load the dataset
df = pd.read_csv('../world_happiness_2021_add.csv')

# List of columns to use as options for x-axis
x_columns = [
    "GDP per capita",
    "Social support",
    "Healthy life expectancy",
    "Freedom to make life choices",
    "Generosity",
    "Perceptions of corruption"
]

# List of countries for the country dropdown, sorted alphabetically
country_options = [{'label': country, 'value': country} for country in sorted(df['Country name'].unique())]

# Columns to be displayed in the parallel coordinates plot
explained_columns = [
    "Explained by: Log GDP per capita",
    "Explained by: Social support",
    "Explained by: Healthy life expectancy",
    "Explained by: Freedom to make life choices",
    "Explained by: Generosity",
    "Explained by: Perceptions of corruption"
]

# Calculate the default axis ranges for each dimension
default_ranges = {col: [df[col].min(), df[col].max()] for col in explained_columns}

# Create a Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("World Happiness 2021"),

    # Div to hold Dropdown and Button in the same row
    html.Div([
        dcc.Dropdown(
            id='x-axis-dropdown',
            options=[{'label': col, 'value': col} for col in x_columns],
            value=x_columns[0],  # Default value
            clearable=False,
            style={'width': 250, 'display': 'inline-block'}
        ),
        dcc.Dropdown(
            id='country-dropdown',
            options=country_options,
            value=None,  # Default is None (all countries)
            placeholder="Select a country",
            clearable=True,
            style={'width': 200, 'display': 'inline-block', 'marginLeft': '10px'}
        ),
        html.Button(
            'Clear Selection',
            id='clear-selection-button',
            n_clicks=0,
            style={'width': 200, 'display': 'inline-block', 'marginLeft': 'auto'}
        ),
    ], style={'display': 'flex', 'width': '100%', 'justify-content': 'space-between'}),

    # Scatter plot
    dcc.Graph(id='scatter-plot'),
    # Parallel Coordinates Plot
    dcc.Graph(id='parallel-coordinates-plot'),

        # Store to hold selected countries
    dcc.Store(id='selected-countries', data=[])
])

# Callback to update both the scatter plot and the parallel coordinates plot
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('parallel-coordinates-plot', 'figure')],
    [Input('x-axis-dropdown', 'value'),
     Input('country-dropdown', 'value'),
     Input('scatter-plot', 'clickData'),
     Input('parallel-coordinates-plot', 'selectedData'),
     Input('clear-selection-button', 'n_clicks')],
    [State('scatter-plot', 'clickData'),
     State('parallel-coordinates-plot', 'selectedData'),
     State('selected-countries', 'data')]
)
def update_plots(selected_x, selected_country, scatter_click, parallel_selected_data, clear_button_clicks, last_scatter_click, last_parallel_selection,selected_countries):
    # Determine what triggered the callback
    ctx = dash.callback_context
    if not ctx.triggered:
        # Default case: show all countries
        filtered_df = df
    else:
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
        # Handle clear selection
        if triggered_id == 'clear-selection-button':
            filtered_df = df
            selected_countries = []
        # Add the country from the dropdown
        elif triggered_id == 'country-dropdown' and selected_country:
            filtered_df = df[df['Country name'] == selected_country]
        # Add the country from the scatter plot click
        elif triggered_id == 'scatter-plot' and scatter_click:
            selected_country = scatter_click['points'][0]['hovertext']
            if selected_country not in selected_countries:
                selected_countries.append(selected_country)
            filtered_df = df[df['Country name'] == selected_country]
        elif triggered_id == 'parallel-coordinates-plot' and parallel_selected_data:
            selected_country = parallel_selected_data['points'][0]['dimensions']['Country name']
            filtered_df = df[df['Country name'] == selected_country]
        else:
            filtered_df = df


    # Create the scatter plot
    scatter_fig = px.scatter(
        df,
        x=selected_x,
        y="Ladder score",
        color="Regional indicator",
        size="Population",
        hover_name="Country name",
        hover_data={  
        "GDP per capita": True,
        "Social support": True,
        "Healthy life expectancy": True,
        "Freedom to make life choices": True,
        "Generosity": True,
        "Perceptions of corruption": True,
        "Ladder score": True,
        "Population": True,
        "Regional indicator": True 
    },
        template='simple_white',
        title=f"Scatter Plot of Ladder Score vs {selected_x}",
        size_max=100
    )
    print(selected_country)

    scatter_fig.update_layout(
        autosize=False,
        width=1200,
        height=600,
        legend=dict(
            orientation="v",
            x=1.1,
            y=0.5,
            xanchor='center',
            yanchor='top'
        ),
        legend_title=dict(
            text='Regional Indicator',
            font=dict(size=16)
        ),
        legend_traceorder='normal'
    )

    # Create the Parallel Coordinates Plot
    parallel_fig = px.parallel_coordinates(
        filtered_df,
        dimensions=explained_columns,
        color="Ladder score",
        color_continuous_scale="rdbu",
        color_continuous_midpoint=5,
        range_color=[2.5, 8], 
        title=f"Contributions to Happiness Score for {selected_country}" if selected_country else "Parallel Coordinates Plot for All Countries",
        template='simple_white'
    )

    # Apply the default axis ranges
    for dim in parallel_fig.data[0]['dimensions']:
        dim['range'] = default_ranges[dim['label']]

    parallel_fig.update_layout(
        autosize=False,
        width=1200,
        height=600
    )

    return scatter_fig, parallel_fig

# Function to stop the server
def shutdown_server():
    import sys
    sys.exit()

# Start a timer to shut down the app after 5 seconds
def schedule_shutdown():
    Timer(5, shutdown_server).start()

# Run the server on a  port 
if __name__ == '__main__':
    schedule_shutdown()
    app.run_server(debug=True, port=8081)

    #this finally has a fix axis for parallel coordinates plot


None
None
India
None
None
